In [1]:
#Import packages
using JuMP, Pkg, CSV, DataFrames, Statistics, Plots, Ipopt, Combinatorics, Distances, LinearAlgebra, AmplNLWriter, NBInclude, Gurobi


In [2]:
@nbinclude("Similarity Factor & Bid-Ask Prices.ipynb")
@nbinclude("Cost Function.ipynb")
@nbinclude("Penalty Function.ipynb")
@nbinclude("Tabu List.ipynb")


In [3]:
#Variable imports
bid_price_df = bid_price()
ask_price_df = ask_price()
similarity = similarityfactor()

4×4 Symmetric{Float64, Matrix{Float64}}:
 0.0            6.4508e5      6.13471e5      6.4578e5
 6.4508e5       0.0       31609.1          700.04
 6.13471e5  31609.1           0.0        32309.1
 6.4578e5     700.04      32309.1            0.0

In [15]:
#Set hyperparameters
mc = 1
mp = 1

#set summed_cost parameter to 0 as initial value
summed_cost = 0;
summed_penalty = 0;

#Initialize our model:
pairs_trading_model = Model(Gurobi.Optimizer)

@variable(pairs_trading_model, x[i= 1:5, j=1:5], Bin)
@objective(pairs_trading_model, Min, costfunct(x, similarity, ask_price_df, bid_price_df))
@constraint(pairs_trading_model, inflow[i in 1:4], sum(x[i,:]) <= 1)
@constraint(pairs_trading_model, outflow[j in 1:4], sum(x[:,j]) <= 1)
@constraint(pairs_trading_model, equality[z in 1:5], sum(x[:,z]) - sum(x[z,:]) == 0)

#dummy constraint
@constraint(pairs_trading_model, dummyin, sum(x[5,:]) == 1)
@constraint(pairs_trading_model, dummyout, sum(x[:,5]) == 1)

optimize!(pairs_trading_model)
solution_summary(pairs_trading_model; verbose = true)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-22
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 5900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 15 rows, 25 columns and 90 nonzeros
Model fingerprint: 0x0418a1f0
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+04, 4e+10]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 0.0000000
Presolve removed 0 rows and 4 columns
Presolve time: 0.00s
Presolved: 15 rows, 21 columns, 82 nonzeros
Variable types: 0 continuous, 21 integer (21 binary)

Root relaxation: objective -4.170309e+10, 6 iterations

* Solver : Gurobi

* Status
  Result count       : 2
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : -4.17031e+10
  Objective bound    : -4.17031e+10
  Relative gap       : 0.00000e+00
  Dual objective value : -4.17031e+10
  Primal solution :
    x[1,1] : 0.00000e+00
    x[1,2] : -0.00000e+00
    x[1,3] : -0.00000e+00
    x[1,4] : 1.00000e+00
    x[1,5] : -0.00000e+00
    x[2,1] : -0.00000e+00
    x[2,2] : 0.00000e+00
    x[2,3] : -0.00000e+00
    x[2,4] : -0.00000e+00
    x[2,5] : -0.00000e+00
    x[3,1] : -0.00000e+00
    x[3,2] : 0.00000e+00
    x[3,3] : 0.00000e+00
    x[3,4] : 0.00000e+00
    x[3,5] : -0.00000e+00
    x[4,1] : -0.00000e+00
    x[4,2] : 0.00000e+00
    x[4,3] : -0.00000e+00
    x[4,4] : 0.00000e+00
    x[4,5] : 1.00000e+00
    x[5,

In [16]:
print(pairs_trading_model)
print(solution_summary(pairs_trading_model))
print(is_solved_and_feasible(pairs_trading_model))

* Solver : Gurobi

* Status
  Result count       : 2
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : -4.17031e+10
  Objective bound    : -4.17031e+10
  Relative gap       : 0.00000e+00
  Dual objective value : -4.17031e+10

* Work counters
  Solve time (sec)   : 9.99928e-04
  Barrier iterations : 0
  Node count         : 1
true